In [71]:
import pandas as pd

In [72]:
cols = [
    'O_D_diff', 'mid_diff', 'GK_O_diff', 'O_GK_diff', 'age_diff', 
    'MV_diff', 'GD_diff', 'top_diff', 'bot_diff', 'form_diff', 'var_diff', 
    'temp', 'weather_desc', 'goal_diff'
]

In [73]:
data_dict = {col: [] for col in cols}

In [74]:
game_data_df = pd.read_csv('../train_data/game_data.csv', index_col=0)
form_data_df = pd.read_csv('../train_data/form_data.csv', index_col=0)
player_data_df = pd.read_csv('../train_data/player_data.csv', index_col=0)